To do:
- Get a new list of apparent magnitudes, now properly including the H band
- Figure out where the BCs come from exactly

# Dataset Build

This notebook will go through my target selection process step by step to make sure the selections and cuts are fully transparent.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from astropy.table import Table

I'll be using the function below to remove duplicates thoughout this code.

In [2]:
def kill_duplicates(df,ID, sep):
    df = df.copy(deep=True)
    sel = df.duplicated(ID, keep=False)
    s = df[sel]
    for idx in range(s[ID].nunique()):
        subset = s[s[ID] == s[ID].values[idx]]
        lock = True  #Iterate and remove the target of largest separation
        while lock:
            if len(subset) > 1.:
                drop = subset[sep].idxmax()
                subset.drop(drop, inplace=True)
                s.drop(drop,inplace=True)
                df.drop(drop, inplace=True)
            elif len(subset == 1.):
                lock = False
            else:
                print('The code shouldnt be here?')
    return df

### 1) Read in the [Kepler x DR2](https://gaia-kepler.fun/) catalogue (thanks Megan Bedell)

In [3]:
data = Table.read('../data/KepxDR2/kepler_dr2_1arcsec.fits', format='fits')
kdf = data.to_pandas()
kdf.rename(columns={'kepid':'KICID'},inplace=True)
print('Targets in Kep x DR2: '+str(len(kdf)))

Targets in Kep x DR2: 195830


### 2) Read in the [Yu et al. 2018](http://adsabs.harvard.edu/abs/2018arXiv180204455Y) catalogue

In [4]:
sfile = '../../Catalogues/RC_catalogues/Yu+18_table1.txt'
yu18_1 = pd.read_csv(sfile, sep='|')

sfile = '../../Catalogues/RC_catalogues/Yu+18_table2.txt'
yu18_2 = pd.read_csv(sfile, sep='|')
yu18 = pd.merge(yu18_1, yu18_2, on='KICID',how='left')
yu18.rename(columns={'EvoPhase':'stage',
                    'err_x':'numax_err',
                    'err.1_x':'dnu_err',
                    'err_y':'Teff_err',
                     'Fe/H':'[Fe/H]',
                    'err.2':'[Fe/H]_err',
                    'logg':'yu_logg',
                    'err.1_y':'yu_logg_err',
                    'err.3_y':'M_err',
                    'err.4_y':'R_err'},inplace=True) #For consistency
print('Targets: '+str(len(yu18)))

Targets: 16094


### 3) Merge the KeplerxDR2 and Yu18 catalogues on KIC

In [5]:
xyu18 = pd.merge(yu18, kdf, on='KICID',how='left')
print('Targets: '+str(len(xyu18)))

Targets: 16135


#### 3.1) Only keep duplicates with the smallest angular separation

In [6]:
xyu18 = kill_duplicates(xyu18, 'KICID', 'kepler_gaia_ang_dist')
print('Targets: '+str(len(xyu18)))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Targets: 16094


#### 3.2) Removing any infinite parallax values

In [7]:
xyu18 = xyu18[np.isfinite(xyu18.parallax)]
print('Targets: '+str(len(xyu18)))

Targets: 16037


#### 3.3) Removing any parallax uncertainties above 35%

In [8]:
xyu18 = xyu18[xyu18.parallax_error/xyu18.parallax < .35]
print('Targets: '+str(len(xyu18)))

Targets: 15822


### 4) Select Red Clump stars only

In [9]:
rcxyu18 = xyu18[xyu18.stage==2]
print('Targets: '+str(len(rcxyu18)))

Targets: 7534


### 5) Add photometry from [2MASS](http://vizier.u-strasbg.fr/cgi-bin/VizieR?-source=B/2mass)
I'll also remove any data that have negative or otherwise unphysical magnitudes or uncertainties.

In addition to the 2MASS photometry, we also use the measure of magnitude in the *Gaia* *G* Band provided with the *Gaia* sample.

#### 5.1) Read in the data

In [21]:
twomass = pd.read_csv('../data/KepxDR2/asu.tsv',sep='|',skiprows=52)
twomass['KICID'] = ''
twomass['KICID'] = twomass['_1'].apply(lambda x: x[4:]).str.strip()
twomass = twomass[:-1] #Delete broken final column
print('Targets in 2MASS: '+str(len(twomass)))

Targets in 2MASS: 7727


#### 5.2) Remove strange string artefacts and NaN values

In [22]:
#Kill NaN values for J band mag err (and by association J band mag)
twomass['e_Jmag'] = twomass['e_Jmag'].str.strip()
twomass.drop(twomass[twomass.e_Jmag == ''].index.values, inplace=True)
twomass['e_Jmag'] = twomass.e_Jmag.astype(float)

#Kill NaN values for K band mag err (and by association K band mag)
twomass.drop(twomass[~np.isfinite(twomass.e_Kmag)].index.values, inplace=True)

#Kill NaN values forHK band mag err (and by association K band mag)
twomass['e_Hmag'] = twomass['e_Hmag'].str.strip()
twomass.drop(twomass[twomass.e_Hmag == ''].index.values, inplace=True)
print('Targets in 2MASS: '+str(len(twomass)))

Targets in 2MASS: 7684


#### 5.2) Remove duplicates on angular distance to target

In [23]:
twomass = kill_duplicates(twomass, 'KICID', '_r')
print('Targets in 2MASS: '+str(len(twomass)))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Targets in 2MASS: 7629


#### 5.3) Merge with our data

In [13]:
rcxyu18['KICID'] = rcxyu18['KICID'].astype(str)
rcxyu18 = pd.merge(rcxyu18, twomass, on='KICID',how='left')
print('Targets: '+str(len(rcxyu18)))

Targets: 7534


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### 6) Get reddening & extinction from [Bayestar 17](http://argonaut.skymaps.info/) ([Green et al. 2018](http://adsabs.harvard.edu/abs/2018AAS...23135002G))

We use the [Bailer-Jones estimated distance](https://arxiv.org/abs/1804.10121) to get a measure of the reddening. Any changes in reddening due to using this value over, say, using 1/$\varpi$, falls within the priors on extinction in our model.

For extinction coefficients, please see references on [omnitool.literature_values](https://github.com/ojhall94/omnitool/blob/master/omnitool/literature_values.py)

In [14]:
from omnitool import spyglass
from omnitool.literature_values import Av_coeffs
sg = spyglass()
sg.pass_position(rcxyu18.ra, rcxyu18.dec, frame='icrs')
sg.pass_distance(rcxyu18.r_est)
rcxyu18['Ebv'] = sg.get_Ebv()
rcxyu18['Aks'] = rcxyu18.Ebv * Av_coeffs['Ks'].values[0]
rcxyu18['Aj'] = rcxyu18.Ebv * Av_coeffs['J'].values[0]
rcxyu18['Ah'] = rcxyu18.Ebv * Av_coeffs['H'].values[0]
rcxyu18['Ag'] = rcxyu18.Ebv * Av_coeffs['G'].values[0]

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 7) Calculate (a *basic* value for) the asteroseismic absolute magnitude
We'll do this using my omnitool package to run the asteroseismic scaling relations (no corrections to them for now, hence the basic).

Bolometric Corrections are taken using the method by Casagrande et al. (??)[**SOURCE?**]

#### 7.1) Get the asteroseismic bolometric magnitude

In [15]:
from omnitool.literature_values import Rsol, Msol, Lsol, Zsol
from omnitool import scalings

sc = scalings(rcxyu18, rcxyu18.numax, rcxyu18.dnu, rcxyu18. Teff,\
                      _numax_err = rcxyu18.numax_err, _dnu_err = rcxyu18.dnu_err,\
                      _Teff_err = rcxyu18.Teff_err)
rcxyu18['R'] = sc.get_radius()/Rsol
rcxyu18['R_err'] = sc.get_radius_err()/Rsol
rcxyu18['M'] = sc.get_mass()/Msol
rcxyu18['M_err'] = sc.get_mass_err()/Msol
rcxyu18['logg'] = sc.get_logg()
rcxyu18['logg_err'] = sc.get_logg_err()
rcxyu18['L'] = sc.get_luminosity()/Lsol
rcxyu18['L_err'] = sc.get_luminosity_err()/Lsol
rcxyu18['Mbol'] = sc.get_bolmag()
rcxyu18['Mbol_err'] = sc.get_bolmag_err()
rcxyu18['Z'] = Zsol * 10 ** rcxyu18['[Fe/H]'].values

#### 7.2) Read in the Casagrande Bolometric Corrections
We'll make sure to kill any duplicates on angular distance before merging

In [16]:
bcdf = pd.read_csv('../data/KepxDR2/casagrande_bcs.csv')
bcdf['KICID'] = bcdf['KICID'].astype(str)
bcdf = kill_duplicates(bcdf, 'KICID', 'kepler_gaia_ang_dist')
rcxyu18 = pd.merge(rcxyu18, bcdf, on='KICID',how='left')
print('Targets: '+str(len(rcxyu18)))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Targets: 7534


#### 7.3) Calculate the absolute magnitudes in J, H and Ks

We assume the intrinsic error on the bolometric correction to be of the order of $0.02\rm mag$

In [17]:
err_bc = 0.02#mag
rcxyu18['ast_MKs'] = rcxyu18.Mbol - rcxyu18.BC_K
rcxyu18['ast_MH'] = rcxyu18.Mbol - rcxyu18.BC_H
rcxyu18['ast_MJ'] = rcxyu18.Mbol - rcxyu18.BC_J
rcxyu18['ast_MG'] = rcxyu18.Mbol - rcxyu18.BC_GAIA
rcxyu18['ast_M_err'] = np.sqrt(rcxyu18.Mbol_err**2 + err_bc**2)

### Final count of targets in ours sample:

In [18]:
print(str(len(rcxyu18)))

7534
